# LangChain 토큰 사용량 확인 가이드

이 노트북에서는 LangChain과 Gemini 모델을 사용할 때 토큰 사용량을 확인하고 관리하는 방법을 배웁니다.

## 목차
1. 환경 설정 및 LangSmith 연결
2. 기본 토큰 사용량 확인
3. `get_num_tokens()` 메서드 사용법
4. 콜백을 통한 토큰 추적
5. LangSmith를 통한 토큰 모니터링
6. 배치 처리 시 토큰 사용량
7. 토큰 사용량 최적화 팁

## 토큰이란?

**토큰(Token)**은 AI 모델이 텍스트를 처리하는 기본 단위입니다:
- **영어**: 보통 1개 단어 = 1~2개 토큰
- **한국어**: 보통 1개 음절 = 1개 토큰
- **특수문자와 공백**도 토큰으로 계산

## 토큰 관리의 중요성

1. **비용 최적화**: 대부분의 AI API는 토큰 수에 따라 요금 부과
2. **성능 향상**: 토큰 수가 적을수록 응답 속도 향상
3. **모델 제한**: 각 모델마다 최대 토큰 제한이 있음
4. **리소스 관리**: 토큰 사용량을 모니터링하여 효율적인 사용

## 1. 환경 설정 및 LangSmith 연결

먼저 필요한 패키지를 임포트하고 LangSmith를 연결합니다.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.callbacks import BaseCallbackHandler
import time

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수 확인
print("🔍 환경 변수 확인 중...")

# Google API 키 확인
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("✅ Google API Key: 설정됨")
else:
    print("⚠️  Google API Key가 설정되지 않았습니다.")

# OpenAI API 키 확인 (대안)
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("✅ OpenAI API Key: 설정됨")
else:
    print("⚠️  OpenAI API Key가 설정되지 않았습니다.")

if not google_api_key and not openai_api_key:
    print("❌ 사용 가능한 API 키가 없습니다.")
    print("   .env 파일에 GOOGLE_API_KEY 또는 OPENAI_API_KEY를 설정하세요.")
    print("   시뮬레이션 모드로 진행됩니다.")

# LangSmith API 키 확인 및 설정
langsmith_api_key = os.getenv("LANGCHAIN_API_KEY")
if langsmith_api_key:
    print("✅ LangSmith API Key: 설정됨")
    # LangSmith 설정
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "CH04-Models"  # 프로젝트 이름 설정
    
    print(f"📊 LangSmith 프로젝트: {os.environ['LANGCHAIN_PROJECT']}")
    print(f"🔄 LangSmith 추적 활성화: {os.environ['LANGCHAIN_TRACING_V2']}")
    print("🌐 LangSmith 대시보드: https://smith.langchain.com/")
else:
    print("⚠️  LangSmith API Key가 설정되지 않았습니다.")
    print("   토큰 추적은 콜백을 통해서만 가능합니다.")

print("\n✅ 환경 설정 완료!")

In [ ]:
# 모델 초기화 - 우선순위: OpenAI > Gemini > 시뮬레이션
llm = None
model_type = "simulation"

# 1. OpenAI 모델 시도 (추천)
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    try:
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            max_tokens=200
        )
        print("🤖 OpenAI GPT-3.5-turbo 모델 사용!")
        model_type = "openai"
        
        # 간단한 테스트
        print("🧪 모델 연결 테스트...")
        test_response = llm.invoke("Hi")
        print(f"✅ OpenAI 모델 테스트 성공!")
        
    except Exception as e:
        print(f"⚠️  OpenAI 모델 초기화 실패: {e}")
        llm = None

# 2. Gemini 모델 시도 (OpenAI가 실패한 경우)
if llm is None:
    google_api_key = os.getenv("GOOGLE_API_KEY")
    if google_api_key:
        try:
            llm = ChatGoogleGenerativeAI(
                model="gemini-1.5-flash",
                temperature=0.7,
                max_output_tokens=200
            )
            print("🤖 Gemini 모델 사용!")
            model_type = "gemini"
            
            # 간단한 테스트 (할당량 초과 시 건너뛰기)
            print("🧪 모델 연결 테스트...")
            test_response = llm.invoke("Hi")
            print(f"✅ Gemini 모델 테스트 성공!")
            
        except Exception as e:
            print(f"⚠️  Gemini 모델 사용 불가: {str(e)[:100]}...")
            if "quota" in str(e).lower() or "429" in str(e):
                print("   → API 할당량 초과로 인한 오류입니다.")
            llm = None

# 3. 시뮬레이션 모드 (모든 API가 실패한 경우)
if llm is None:
    print("🔧 시뮬레이션 모드로 전환!")
    print("   → 실제 LLM 호출 없이 토큰 계산 및 데모를 진행합니다.")
    model_type = "simulation"
    
    # 시뮬레이션용 가짜 LLM 클래스
    class SimulationLLM:
        def __init__(self):
            self.model = "simulation-model"
            self.temperature = 0.7
            
        def get_num_tokens(self, text):
            # 대략적인 토큰 계산 (영어: 4문자당 1토큰, 한국어: 1문자당 1토큰)
            korean_chars = len([c for c in text if ord(c) > 127])
            english_chars = len(text) - korean_chars
            return korean_chars + (english_chars // 4)
        
        def invoke(self, messages):
            # 시뮬레이션 응답
            class SimResponse:
                def __init__(self):
                    self.content = "시뮬레이션 모드 응답입니다. 실제 LLM은 호출되지 않았습니다."
            return SimResponse()
    
    llm = SimulationLLM()

# 모델 정보 출력
print(f"\n📋 최종 모델 설정:")
print(f"   🤖 모델: {llm.model if hasattr(llm, 'model') else 'Unknown'}")
print(f"   🌡️  온도: {llm.temperature if hasattr(llm, 'temperature') else 'Unknown'}")
print(f"   🔧 모드: {model_type}")

if model_type == "simulation":
    print(f"\n💡 시뮬레이션 모드 안내:")
    print(f"   • 토큰 계산 기능은 정상 작동합니다")
    print(f"   • LLM 호출이 필요한 셀은 시뮬레이션 데이터를 사용합니다")
    print(f"   • 실제 API 비용이 발생하지 않습니다")

print(f"\n✅ 모델 초기화 완료!")

## 2. 기본 토큰 사용량 확인

LangChain에서 토큰 사용량을 확인하는 가장 기본적인 방법들을 알아봅시다.

### 토큰 계산 방법
1. **`get_num_tokens()`**: 텍스트의 토큰 수 미리 계산
2. **응답 메타데이터**: 실제 API 호출 후 사용된 토큰 확인
3. **콜백 핸들러**: 실시간 토큰 추적

In [ ]:
############################################################################
# 응답 메타데이터를 통한 토큰 사용량 확인
############################################################################

print("📊 응답 메타데이터를 통한 토큰 사용량 확인")
print("=" * 50)

# 다양한 길이의 질문으로 테스트
test_questions = [
    "안녕하세요!",
    "Python 프로그래밍의 장점 3가지를 알려주세요.",
    "인공지능과 머신러닝의 차이점을 상세히 설명하고, 각각의 활용 분야와 미래 전망에 대해 설명해주세요."
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. 질문: {question}")
    print(f"   질문 길이: {len(question)} 문자")
    
    # 메시지 생성
    message = HumanMessage(content=question)
    
    # API 호출
    response = llm.invoke([message])
    
    # 응답 정보
    print(f"   응답: {response.content[:50]}...")
    print(f"   응답 길이: {len(response.content)} 문자")
    
    # 메타데이터에서 토큰 정보 확인
    if hasattr(response, 'response_metadata'):
        metadata = response.response_metadata
        print(f"   📈 메타데이터: {metadata}")
        
        # 토큰 사용량 정보 추출
        if 'usage_metadata' in metadata:
            usage = metadata['usage_metadata']
            print(f"   🎯 입력 토큰: {usage.get('prompt_tokens', 'N/A')}")
            print(f"   🎯 출력 토큰: {usage.get('completion_tokens', 'N/A')}")
            print(f"   🎯 총 토큰: {usage.get('total_tokens', 'N/A')}")
        else:
            print(f"   ⚠️  토큰 정보가 메타데이터에 없습니다.")
    else:
        print(f"   ⚠️  응답에 메타데이터가 없습니다.")

print("\n✅ 메타데이터 토큰 확인 완료!")

## 3. `get_num_tokens()` 메서드 사용법

`get_num_tokens()` 메서드를 사용하면 실제 API를 호출하기 전에 텍스트의 토큰 수를 미리 계산할 수 있습니다.

### 주요 활용 사례
- **비용 예측**: API 호출 전 비용 계산
- **토큰 제한 확인**: 모델의 최대 토큰 제한 준수
- **프롬프트 최적화**: 토큰 수를 고려한 프롬프트 설계

In [ ]:
############################################################################
# get_num_tokens() 메서드를 사용한 토큰 수 계산
############################################################################

print("🔢 get_num_tokens() 메서드를 사용한 토큰 계산")
print("=" * 50)

# 다양한 텍스트로 테스트
test_texts = [
    "안녕하세요!",
    "Hello, world!",
    "Python은 배우기 쉽고 강력한 프로그래밍 언어입니다.",
    "LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발을 위한 프레임워크입니다. 다양한 컴포넌트를 제공하여 복잡한 AI 워크플로우를 쉽게 구축할 수 있습니다.",
    """
    긴 텍스트 예제입니다. 
    여러 줄에 걸친 텍스트의 토큰 수를 계산해봅시다.
    토큰 계산은 모델마다 다를 수 있으며,
    한국어와 영어의 토큰 계산 방식도 다릅니다.
    이런 차이점을 이해하는 것이 중요합니다.
    """
]

print("📝 텍스트별 토큰 수 분석:")
for i, text in enumerate(test_texts, 1):
    # 토큰 수 계산
    try:
        token_count = llm.get_num_tokens(text)
        char_count = len(text)
        
        print(f"\n{i}. 텍스트: {text[:50]}{'...' if len(text) > 50 else ''}")
        print(f"   📏 문자 수: {char_count}")
        print(f"   🎯 토큰 수: {token_count}")
        print(f"   📊 토큰/문자 비율: {token_count/char_count:.2f}")
        
    except Exception as e:
        print(f"\n{i}. 텍스트: {text[:50]}{'...' if len(text) > 50 else ''}")
        print(f"   ❌ 토큰 계산 실패: {e}")

# 메시지 객체의 토큰 수 계산
print(f"\n🔍 메시지 객체의 토큰 수 계산:")
messages = [
    SystemMessage(content="당신은 도움이 되는 AI 어시스턴트입니다."),
    HumanMessage(content="Python 프로그래밍에 대해 간단히 설명해주세요.")
]

try:
    total_tokens = llm.get_num_tokens_from_messages(messages)
    print(f"   📨 전체 메시지 토큰 수: {total_tokens}")
    
    for i, msg in enumerate(messages):
        individual_tokens = llm.get_num_tokens(msg.content)
        print(f"   📩 메시지 {i+1} ({type(msg).__name__}): {individual_tokens} 토큰")
        
except Exception as e:
    print(f"   ❌ 메시지 토큰 계산 실패: {e}")
    # 개별 메시지로 계산 시도
    total = 0
    for i, msg in enumerate(messages):
        try:
            tokens = llm.get_num_tokens(msg.content)
            total += tokens
            print(f"   📩 메시지 {i+1} ({type(msg).__name__}): {tokens} 토큰")
        except Exception as e2:
            print(f"   ❌ 메시지 {i+1} 계산 실패: {e2}")
    print(f"   📨 개별 계산 총합: {total} 토큰")

print("\n✅ get_num_tokens() 실습 완료!")

## 4. 콜백을 통한 토큰 추적

콜백 핸들러를 사용하면 실시간으로 토큰 사용량을 추적하고 로깅할 수 있습니다.

### 콜백의 장점
- **실시간 모니터링**: API 호출 중 토큰 사용량 실시간 확인
- **세부 추적**: 각 단계별 토큰 사용량 상세 분석
- **커스텀 로깅**: 원하는 형태로 토큰 정보 기록
- **비용 관리**: 실시간 비용 계산 및 알림

In [ ]:
############################################################################
# 토큰 추적을 위한 커스텀 콜백 핸들러 구현
############################################################################

from langchain_core.callbacks import BaseCallbackHandler
from typing import Any, Dict, List
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult

class TokenUsageCallback(BaseCallbackHandler):
    """토큰 사용량을 추적하는 커스텀 콜백 핸들러"""
    
    def __init__(self):
        self.total_tokens = 0
        self.total_prompt_tokens = 0
        self.total_completion_tokens = 0
        self.call_count = 0
        self.cost_per_1k_tokens = 0.001  # Gemini Flash 예상 비용 (USD)
        self.calls_history = []
    
    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs) -> None:
        """LLM 호출 시작 시 실행"""
        self.call_count += 1
        print(f"\n🚀 LLM 호출 #{self.call_count} 시작")
        print(f"   📝 프롬프트 수: {len(prompts)}")
        
        # 프롬프트의 예상 토큰 수 계산 (가능한 경우)
        for i, prompt in enumerate(prompts):
            print(f"   📄 프롬프트 {i+1}: {prompt[:50]}...")
    
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        """LLM 호출 완료 시 실행"""
        print(f"✅ LLM 호출 #{self.call_count} 완료")
        
        # 토큰 사용량 정보 추출
        if response.llm_output:
            usage = response.llm_output.get('usage_metadata', {})
            
            prompt_tokens = usage.get('prompt_tokens', 0)
            completion_tokens = usage.get('completion_tokens', 0)
            total_tokens_call = usage.get('total_tokens', 0)
            
            # 누적 통계 업데이트
            self.total_prompt_tokens += prompt_tokens
            self.total_completion_tokens += completion_tokens
            self.total_tokens += total_tokens_call
            
            # 이번 호출 정보 저장
            call_info = {
                'call_number': self.call_count,
                'prompt_tokens': prompt_tokens,
                'completion_tokens': completion_tokens,
                'total_tokens': total_tokens_call,
                'timestamp': time.time()
            }
            self.calls_history.append(call_info)
            
            # 상세 정보 출력
            print(f"   🎯 이번 호출 토큰:")
            print(f"      입력: {prompt_tokens} 토큰")
            print(f"      출력: {completion_tokens} 토큰")
            print(f"      총합: {total_tokens_call} 토큰")
            
            # 예상 비용 계산
            estimated_cost = (total_tokens_call / 1000) * self.cost_per_1k_tokens
            print(f"   💰 예상 비용: ${estimated_cost:.6f}")
            
        else:
            print(f"   ⚠️  토큰 사용량 정보를 찾을 수 없습니다.")
    
    def on_llm_error(self, error: Exception, **kwargs) -> None:
        """LLM 호출 에러 시 실행"""
        print(f"❌ LLM 호출 #{self.call_count} 에러: {error}")
    
    def get_summary(self):
        """토큰 사용량 요약 정보 반환"""
        total_cost = (self.total_tokens / 1000) * self.cost_per_1k_tokens
        
        summary = {
            'total_calls': self.call_count,
            'total_tokens': self.total_tokens,
            'total_prompt_tokens': self.total_prompt_tokens,
            'total_completion_tokens': self.total_completion_tokens,
            'estimated_total_cost': total_cost,
            'average_tokens_per_call': self.total_tokens / max(self.call_count, 1),
            'calls_history': self.calls_history
        }
        return summary
    
    def print_summary(self):
        """토큰 사용량 요약을 예쁘게 출력"""
        summary = self.get_summary()
        
        print(f"\n📊 토큰 사용량 요약")
        print(f"=" * 40)
        print(f"🔢 총 호출 수: {summary['total_calls']}")
        print(f"🎯 총 토큰 수: {summary['total_tokens']:,}")
        print(f"   ├─ 입력 토큰: {summary['total_prompt_tokens']:,}")
        print(f"   └─ 출력 토큰: {summary['total_completion_tokens']:,}")
        print(f"📊 호출당 평균: {summary['average_tokens_per_call']:.1f} 토큰")
        print(f"💰 예상 총 비용: ${summary['estimated_total_cost']:.6f}")

# 콜백 핸들러 인스턴스 생성
token_callback = TokenUsageCallback()
print("✅ 토큰 추적 콜백 핸들러 생성 완료!")

In [ ]:
############################################################################
# 콜백 핸들러를 사용한 토큰 추적 테스트
############################################################################

print("🧪 콜백 핸들러를 사용한 토큰 추적 테스트")
print("=" * 50)

# 테스트 질문들
test_questions = [
    "안녕하세요!",
    "Python의 주요 특징 3가지를 알려주세요.",
    "머신러닝과 딥러닝의 차이점을 설명하고, 각각의 대표적인 알고리즘을 예를 들어 설명해주세요."
]

# 각 질문에 대해 콜백과 함께 LLM 호출
for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*20} 질문 {i} {'='*20}")
    print(f"❓ 질문: {question}")
    
    # 콜백을 포함하여 LLM 호출
    response = llm.invoke(
        [HumanMessage(content=question)],
        config={"callbacks": [token_callback]}
    )
    
    print(f"💬 응답: {response.content}")
    print(f"📏 응답 길이: {len(response.content)} 문자")

# 전체 토큰 사용량 요약 출력
print(f"\n" + "="*60)
token_callback.print_summary()

# 세부 호출 이력 출력
print(f"\n📋 세부 호출 이력:")
for call in token_callback.calls_history:
    print(f"   호출 #{call['call_number']}: "
          f"입력 {call['prompt_tokens']} + 출력 {call['completion_tokens']} "
          f"= 총 {call['total_tokens']} 토큰")

print("\n✅ 콜백 기반 토큰 추적 테스트 완료!")

## 5. LangSmith를 통한 토큰 모니터링

LangSmith는 LangChain 애플리케이션의 모니터링과 디버깅을 위한 플랫폼입니다. 토큰 사용량도 자동으로 추적됩니다.

### LangSmith의 토큰 추적 기능
- **자동 수집**: 모든 LLM 호출의 토큰 사용량 자동 기록
- **시각화**: 대시보드를 통한 토큰 사용량 시각화
- **비용 분석**: 프로젝트별, 시간별 비용 분석
- **성능 모니터링**: 토큰 효율성 및 최적화 지점 파악

In [ ]:
############################################################################
# LangSmith를 통한 토큰 추적 테스트
############################################################################

print("🌐 LangSmith를 통한 토큰 추적 테스트")
print("=" * 50)

# LangSmith 추적 상태 확인
langsmith_enabled = os.getenv("LANGCHAIN_TRACING_V2") == "true"
project_name = os.getenv("LANGCHAIN_PROJECT", "기본 프로젝트")

print(f"📊 LangSmith 추적: {'✅ 활성화' if langsmith_enabled else '❌ 비활성화'}")
print(f"📁 프로젝트명: {project_name}")

if langsmith_enabled:
    print(f"🌐 대시보드: https://smith.langchain.com/")
    print(f"   → 프로젝트 '{project_name}'에서 실시간 토큰 사용량을 확인할 수 있습니다.")
    
    # LangSmith 추적과 함께 체인 실행
    from langchain_core.prompts import ChatPromptTemplate
    
    print(f"\n🔗 체인을 통한 LangSmith 추적 테스트")
    
    # 간단한 체인 생성
    prompt = ChatPromptTemplate.from_template(
        "다음 주제에 대해 간단히 설명해주세요: {topic}"
    )
    parser = StrOutputParser()
    chain = prompt | llm | parser
    
    # 여러 주제로 체인 실행
    topics = ["인공지능", "블록체인", "양자컴퓨팅"]
    
    for topic in topics:
        print(f"\n🎯 주제: {topic}")
        result = chain.invoke({"topic": topic})
        print(f"📝 응답: {result[:100]}...")
        print(f"✅ LangSmith에 토큰 사용량이 기록되었습니다.")
    
    print(f"\n💡 LangSmith 대시보드에서 다음 정보를 확인하세요:")
    print(f"   • 각 호출의 토큰 사용량")
    print(f"   • 입력/출력 토큰 분석")
    print(f"   • 시간별 사용량 추이")
    print(f"   • 비용 분석 및 예측")
    
else:
    print(f"\n⚠️  LangSmith가 비활성화되어 있습니다.")
    print(f"   환경 변수 LANGCHAIN_API_KEY를 설정하고 LANGCHAIN_TRACING_V2=true로 설정하세요.")
    
    # LangSmith 없이도 기본 추적 시연
    print(f"\n🔧 로컬 토큰 추적으로 대체 실행:")
    
    prompt = ChatPromptTemplate.from_template(
        "다음 주제에 대해 간단히 설명해주세요: {topic}"
    )
    parser = StrOutputParser()
    
    # 새로운 콜백 인스턴스로 체인 테스트
    local_callback = TokenUsageCallback()
    
    topics = ["인공지능", "블록체인"]
    for topic in topics:
        print(f"\n🎯 주제: {topic}")
        
        # 각 단계별로 토큰 추적
        formatted_prompt = prompt.format_messages(topic=topic)
        response = llm.invoke(
            formatted_prompt, 
            config={"callbacks": [local_callback]}
        )
        result = parser.invoke(response)
        
        print(f"📝 응답: {result[:100]}...")
    
    # 로컬 추적 결과 요약
    local_callback.print_summary()

print(f"\n✅ LangSmith 토큰 추적 테스트 완료!")

## 6. 배치 처리 시 토큰 사용량

여러 개의 요청을 배치로 처리할 때 토큰 사용량을 효율적으로 관리하는 방법을 알아봅시다.

### 배치 처리의 장점
- **비용 효율성**: 개별 호출보다 배치 처리가 일반적으로 더 효율적
- **처리 속도**: 동시 처리로 전체 처리 시간 단축
- **토큰 예측**: 전체 배치의 토큰 사용량 미리 계산 가능
- **에러 처리**: 배치 단위로 에러 관리

In [ ]:
############################################################################
# 배치 처리 시 토큰 사용량 관리
############################################################################

print("📦 배치 처리 시 토큰 사용량 관리")
print("=" * 50)

# 배치 처리용 질문 목록
batch_questions = [
    "Python이란 무엇인가요?",
    "머신러닝의 기본 개념을 설명해주세요.",
    "딥러닝과 신경망의 관계는?",
    "자연어 처리의 주요 기술들은?",
    "LangChain의 핵심 기능은?",
    "AI 윤리의 중요성은?",
    "클라우드 컴퓨팅의 장점은?",
    "데이터 과학자의 역할은?"
]

print(f"📋 배치 크기: {len(batch_questions)}개 질문")

# 1. 사전 토큰 계산
print(f"\n1️⃣ 사전 토큰 계산")
print("-" * 30)

total_estimated_tokens = 0
question_tokens = []

for i, question in enumerate(batch_questions, 1):
    try:
        tokens = llm.get_num_tokens(question)
        question_tokens.append(tokens)
        total_estimated_tokens += tokens
        print(f"   질문 {i}: {tokens} 토큰 - {question}")
    except Exception as e:
        print(f"   질문 {i}: 토큰 계산 실패 - {question}")
        question_tokens.append(0)

print(f"\n📊 사전 계산 결과:")
print(f"   총 입력 토큰 (예상): {total_estimated_tokens}")
print(f"   평균 질문 길이: {total_estimated_tokens/len(batch_questions):.1f} 토큰")

# 출력 토큰 예상 (평균적으로 입력의 2-3배 가정)
estimated_output_tokens = total_estimated_tokens * 2.5
estimated_total = total_estimated_tokens + estimated_output_tokens
estimated_cost = (estimated_total / 1000) * 0.001

print(f"   예상 출력 토큰: {estimated_output_tokens:.0f}")
print(f"   예상 총 토큰: {estimated_total:.0f}")
print(f"   예상 비용: ${estimated_cost:.6f}")

# 2. 배치 실행 (콜백과 함께)
print(f"\n2️⃣ 배치 실행 (토큰 추적)")
print("-" * 30)

batch_callback = TokenUsageCallback()
batch_responses = []

start_time = time.time()

for i, question in enumerate(batch_questions, 1):
    print(f"\n⏳ 처리 중: {i}/{len(batch_questions)} - {question[:30]}...")
    
    try:
        response = llm.invoke(
            [HumanMessage(content=question)],
            config={"callbacks": [batch_callback]}
        )
        batch_responses.append({
            'question': question,
            'response': response.content,
            'success': True
        })
        
    except Exception as e:
        print(f"   ❌ 에러: {e}")
        batch_responses.append({
            'question': question,
            'response': None,
            'success': False,
            'error': str(e)
        })

end_time = time.time()
processing_time = end_time - start_time

# 3. 배치 처리 결과 분석
print(f"\n3️⃣ 배치 처리 결과 분석")
print("-" * 30)

successful_responses = [r for r in batch_responses if r['success']]
failed_responses = [r for r in batch_responses if not r['success']]

print(f"✅ 성공: {len(successful_responses)}개")
print(f"❌ 실패: {len(failed_responses)}개")
print(f"⏱️  총 처리 시간: {processing_time:.2f}초")
print(f"📊 평균 처리 시간: {processing_time/len(batch_questions):.2f}초/질문")

# 토큰 사용량 요약
batch_callback.print_summary()

# 실제 vs 예상 비교
actual_total = batch_callback.total_tokens
prediction_accuracy = (1 - abs(estimated_total - actual_total) / actual_total) * 100

print(f"\n📈 예상 vs 실제 비교:")
print(f"   예상 총 토큰: {estimated_total:.0f}")
print(f"   실제 총 토큰: {actual_total}")
print(f"   예측 정확도: {prediction_accuracy:.1f}%")

# 4. 응답 샘플 출력
print(f"\n4️⃣ 응답 샘플 (처음 3개)")
print("-" * 30)

for i, response in enumerate(successful_responses[:3], 1):
    print(f"\n질문 {i}: {response['question']}")
    print(f"응답: {response['response'][:100]}...")

print(f"\n✅ 배치 처리 토큰 관리 완료!")

## 7. 토큰 사용량 최적화 팁

토큰 사용량을 최적화하여 비용을 절약하고 성능을 향상시키는 실용적인 방법들을 알아봅시다.

### 최적화 전략
1. **프롬프트 최적화**: 불필요한 단어 제거, 명확한 지시
2. **응답 길이 제한**: `max_output_tokens` 설정으로 출력 제한
3. **시스템 메시지 활용**: 반복적인 컨텍스트를 시스템 메시지로 분리
4. **캐싱 활용**: 동일한 요청의 재사용
5. **배치 처리**: 여러 요청을 효율적으로 묶어서 처리

In [ ]:
############################################################################
# 토큰 사용량 최적화 실습
############################################################################

print("⚡ 토큰 사용량 최적화 실습")
print("=" * 50)

# 1. 프롬프트 최적화 비교
print("1️⃣ 프롬프트 최적화 비교")
print("-" * 30)

# 비효율적인 프롬프트 vs 최적화된 프롬프트
inefficient_prompt = """
안녕하세요! 저는 현재 Python 프로그래밍 언어에 대해서 배우고 있는 학생입니다. 
Python이라는 프로그래밍 언어가 다른 프로그래밍 언어들과 비교했을 때 어떤 장점들이 있는지, 
그리고 어떤 분야에서 주로 사용되고 있는지에 대해서 자세하고 친절하게 설명해주시면 감사하겠습니다. 
가능하면 예시도 함께 들어주세요.
"""

efficient_prompt = "Python의 주요 장점 3가지와 활용 분야를 간단히 설명해주세요."

# 토큰 수 비교
inefficient_tokens = llm.get_num_tokens(inefficient_prompt.strip())
efficient_tokens = llm.get_num_tokens(efficient_prompt)

print(f"❌ 비효율적 프롬프트:")
print(f"   내용: {inefficient_prompt.strip()[:50]}...")
print(f"   토큰 수: {inefficient_tokens}")

print(f"\n✅ 최적화된 프롬프트:")
print(f"   내용: {efficient_prompt}")
print(f"   토큰 수: {efficient_tokens}")

print(f"\n📊 최적화 결과:")
print(f"   토큰 절약: {inefficient_tokens - efficient_tokens} 토큰")
print(f"   절약률: {((inefficient_tokens - efficient_tokens) / inefficient_tokens * 100):.1f}%")

# 2. 응답 길이 제한 비교 (시뮬레이션)
print(f"\n2️⃣ 응답 길이 제한 비교 (시뮬레이션)")
print("-" * 30)

question = "인공지능의 미래에 대해 설명해주세요."

# 제한 없는 모델 설정
unlimited_llm_config = {
    "model": "gemini-1.5-flash",
    "temperature": 0.7,
    "max_output_tokens": None  # 제한 없음
}

# 제한 있는 모델 설정
limited_llm_config = {
    "model": "gemini-1.5-flash", 
    "temperature": 0.7,
    "max_output_tokens": 100  # 출력 제한
}

print(f"질문: {question}")
print(f"입력 토큰: {llm.get_num_tokens(question)}")

# API 호출 대신 시뮬레이션으로 대체
print(f"\n🔓 제한 없는 응답 (시뮬레이션):")
print(f"설정: {unlimited_llm_config}")
estimated_unlimited_output = 150  # 예상 출력 토큰
unlimited_total_estimated = llm.get_num_tokens(question) + estimated_unlimited_output
print(f"예상 출력 토큰: {estimated_unlimited_output}")
print(f"예상 총 토큰: {unlimited_total_estimated}")

print(f"\n🔒 길이 제한 응답 (시뮬레이션):")
print(f"설정: {limited_llm_config}")
estimated_limited_output = 100  # 제한된 출력 토큰
limited_total_estimated = llm.get_num_tokens(question) + estimated_limited_output
print(f"예상 출력 토큰: {estimated_limited_output}")
print(f"예상 총 토큰: {limited_total_estimated}")

# 결과 비교
print(f"\n📊 응답 길이 제한 효과 (시뮬레이션):")
print(f"   제한 없음: {unlimited_total_estimated} 토큰")
print(f"   제한 있음: {limited_total_estimated} 토큰")
print(f"   절약: {unlimited_total_estimated - limited_total_estimated} 토큰")

if unlimited_total_estimated > 0:
    savings_percent = ((unlimited_total_estimated - limited_total_estimated) / unlimited_total_estimated) * 100
    print(f"   절약률: {savings_percent:.1f}%")

print(f"\n💡 실제 환경에서는 max_output_tokens 설정으로 출력 토큰을 제한할 수 있습니다.")

# 3. 시스템 메시지 활용
print(f"\n3️⃣ 시스템 메시지 활용")
print("-" * 30)

# 반복적인 컨텍스트가 포함된 질문들
questions_with_context = [
    "당신은 Python 전문가입니다. Python의 리스트 comprehension에 대해 설명해주세요.",
    "당신은 Python 전문가입니다. Python의 데코레이터에 대해 설명해주세요.",
    "당신은 Python 전문가입니다. Python의 제너레이터에 대해 설명해주세요."
]

# 시스템 메시지 + 간단한 질문들
system_message = SystemMessage(content="당신은 Python 전문가입니다.")
optimized_questions = [
    "리스트 comprehension에 대해 설명해주세요.",
    "데코레이터에 대해 설명해주세요.", 
    "제너레이터에 대해 설명해주세요."
]

# 토큰 수 계산
context_method_tokens = sum(llm.get_num_tokens(q) for q in questions_with_context)
system_method_tokens = llm.get_num_tokens(system_message.content) + sum(llm.get_num_tokens(q) for q in optimized_questions)

print(f"❌ 반복 컨텍스트 방식:")
print(f"   총 입력 토큰: {context_method_tokens}")

print(f"\n✅ 시스템 메시지 방식:")
print(f"   시스템 메시지: {llm.get_num_tokens(system_message.content)} 토큰")
print(f"   질문들: {sum(llm.get_num_tokens(q) for q in optimized_questions)} 토큰")
print(f"   총 입력 토큰: {system_method_tokens}")

print(f"\n📊 시스템 메시지 최적화 결과:")
print(f"   토큰 절약: {context_method_tokens - system_method_tokens} 토큰")
print(f"   절약률: {((context_method_tokens - system_method_tokens) / context_method_tokens * 100):.1f}%")

print(f"\n✅ 토큰 최적화 실습 완료!")

## 정리 및 모범 사례

이 노트북에서 배운 토큰 사용량 확인 및 관리 방법들을 정리해봅시다.

### 🎯 주요 학습 내용

1. **응답 메타데이터**: `response.response_metadata`에서 토큰 정보 확인
2. **`get_num_tokens()`**: 사전 토큰 계산으로 비용 예측
3. **콜백 핸들러**: 실시간 토큰 추적 및 상세 분석
4. **LangSmith**: 자동 토큰 모니터링 및 시각화
5. **배치 처리**: 대량 요청의 효율적인 토큰 관리
6. **최적화 기법**: 프롬프트, 응답 길이, 시스템 메시지 최적화

### 📋 토큰 관리 체크리스트

| 단계 | 확인 사항 | 도구/방법 |
|------|-----------|-----------|
| **사전 계획** | 예상 토큰 수 계산 | `get_num_tokens()` |
| **실행 중** | 실시간 토큰 추적 | 콜백 핸들러 |
| **모니터링** | 장기적 사용량 분석 | LangSmith |
| **최적화** | 토큰 효율성 개선 | 프롬프트 최적화 |

### ⚠️ 주의사항

1. **정확성**: 토큰 계산은 모델마다 다를 수 있음
2. **비용 예측**: 실제 비용은 API 제공자의 정책에 따라 변동
3. **성능 영향**: 과도한 토큰 추적은 성능에 영향을 줄 수 있음
4. **메타데이터 의존성**: 모든 모델이 동일한 메타데이터를 제공하지 않음

### 🚀 실전 활용 팁

1. **개발 단계**: 콜백을 활용한 상세 토큰 분석
2. **테스트 단계**: 배치 처리로 토큰 사용량 검증
3. **프로덕션**: LangSmith로 지속적인 모니터링
4. **최적화**: 정기적인 프롬프트 및 설정 최적화

### 💡 비용 절약 요령

- **짧고 명확한 프롬프트** 사용
- **응답 길이 제한** 설정 (`max_output_tokens`)
- **시스템 메시지** 활용으로 반복 컨텍스트 제거
- **캐싱** 및 **배치 처리** 적극 활용
- **정기적인 토큰 사용량 분석** 및 최적화

In [ ]:
############################################################################
# 종합 예제: 토큰 사용량 모니터링 시스템
############################################################################

print("🎯 종합 예제: 토큰 사용량 모니터링 시스템")
print("=" * 50)

class TokenMonitor:
    """토큰 사용량을 종합적으로 모니터링하는 클래스"""
    
    def __init__(self, llm, cost_per_1k_tokens=0.001):
        self.llm = llm
        self.cost_per_1k_tokens = cost_per_1k_tokens
        self.callback = TokenUsageCallback()
        self.sessions = []
    
    def estimate_tokens(self, text):
        """텍스트의 토큰 수 추정"""
        try:
            return self.llm.get_num_tokens(text)
        except:
            return len(text) // 4  # 대략적인 추정
    
    def process_with_monitoring(self, messages, session_name="기본 세션"):
        """토큰 모니터링과 함께 메시지 처리"""
        print(f"\n🔄 세션 시작: {session_name}")
        
        # 사전 토큰 추정
        if isinstance(messages, str):
            messages = [HumanMessage(content=messages)]
        elif isinstance(messages, list) and isinstance(messages[0], str):
            messages = [HumanMessage(content=msg) for msg in messages]
        
        estimated_input = sum(self.estimate_tokens(msg.content) for msg in messages)
        print(f"   📊 예상 입력 토큰: {estimated_input}")
        
        # 실제 처리
        session_callback = TokenUsageCallback()
        
        try:
            response = self.llm.invoke(
                messages, 
                config={"callbacks": [session_callback]}
            )
            
            # 세션 정보 저장
            session_info = {
                'name': session_name,
                'estimated_input': estimated_input,
                'actual_tokens': session_callback.total_tokens,
                'response': response.content,
                'success': True
            }
            self.sessions.append(session_info)
            
            print(f"   ✅ 처리 완료")
            print(f"   📈 실제 사용 토큰: {session_callback.total_tokens}")
            print(f"   💰 세션 비용: ${(session_callback.total_tokens/1000)*self.cost_per_1k_tokens:.6f}")
            
            return response.content
            
        except Exception as e:
            session_info = {
                'name': session_name,
                'estimated_input': estimated_input, 
                'actual_tokens': 0,
                'response': None,
                'success': False,
                'error': str(e)
            }
            self.sessions.append(session_info)
            print(f"   ❌ 처리 실패: {e}")
            return None
    
    def get_summary_report(self):
        """종합 리포트 생성"""
        if not self.sessions:
            return "아직 처리된 세션이 없습니다."
        
        total_sessions = len(self.sessions)
        successful_sessions = [s for s in self.sessions if s['success']]
        failed_sessions = [s for s in self.sessions if not s['success']]
        
        total_tokens = sum(s['actual_tokens'] for s in successful_sessions)
        total_cost = (total_tokens / 1000) * self.cost_per_1k_tokens
        
        accuracy_data = [(s['estimated_input'], s['actual_tokens']) for s in successful_sessions if s['actual_tokens'] > 0]
        avg_accuracy = 0
        if accuracy_data:
            accuracies = [1 - abs(est - act) / act for est, act in accuracy_data if act > 0]
            avg_accuracy = sum(accuracies) / len(accuracies) * 100
        
        report = f"""
📊 토큰 사용량 종합 리포트
{'='*40}
🔢 총 세션 수: {total_sessions}개
   ├─ 성공: {len(successful_sessions)}개
   └─ 실패: {len(failed_sessions)}개

🎯 토큰 사용량:
   ├─ 총 토큰: {total_tokens:,}개
   ├─ 평균/세션: {total_tokens/max(len(successful_sessions), 1):.1f}개
   └─ 예측 정확도: {avg_accuracy:.1f}%

💰 비용 정보:
   ├─ 총 비용: ${total_cost:.6f}
   └─ 평균/세션: ${total_cost/max(len(successful_sessions), 1):.6f}

📋 세션별 상세:"""
        
        for i, session in enumerate(self.sessions[-5:], 1):  # 최근 5개만
            status = "✅" if session['success'] else "❌"
            tokens = session['actual_tokens'] if session['success'] else 0
            report += f"\n   {status} {session['name']}: {tokens} 토큰"
        
        if len(self.sessions) > 5:
            report += f"\n   ... (총 {len(self.sessions)}개 세션)"
        
        return report

# 토큰 모니터 인스턴스 생성
monitor = TokenMonitor(llm)

# 다양한 세션으로 테스트
test_sessions = [
    ("간단한 질문", "안녕하세요!"),
    ("중간 길이 질문", "Python 프로그래밍의 장점을 3가지 알려주세요."),
    ("복잡한 질문", "머신러닝과 딥러닝의 차이점을 상세히 설명하고, 각각의 활용 분야와 미래 전망에 대해 논의해주세요.")
]

print("🧪 다양한 세션으로 토큰 모니터링 테스트")

for session_name, question in test_sessions:
    response = monitor.process_with_monitoring(question, session_name)
    if response:
        print(f"   📝 응답: {response[:50]}...")

# 종합 리포트 출력
print(monitor.get_summary_report())

print(f"\n🎉 토큰 사용량 확인 가이드 완료!")
print(f"💡 이제 여러분은 LangChain에서 토큰을 효율적으로 관리할 수 있습니다!")